In [3]:
import os
import requests
import json

from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.application  import MIMEApplication
from email.encoders import encode_noop

os.chdir('/home/jovyan/work')
import init
import do_calibration
import secrets
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
R = requests.post('http://nova.astrometry.net/api/login', data={'request-json': json.dumps({"apikey": secrets.astrometrynet_api})})
sessionkey=R.json().get("session")
print(R.text)
_, filename, reference_frame_index = do_calibration.get_reference_frame(100)
filedata=open(filename, 'rb')
files={'file': filedata}
jsondata={"session": sessionkey, "publicly_visible": "n", "allow_commercial_use": "n"}
uploadurl='http://nova.astrometry.net/api/url_upload'


{"status": "success", "message": "authenticated user: mike.rosseel@gmail.com", "session": "r1m2bolrpj1gklic5o767v61811ferhr"}
Reference frame: kout000069.fts, index: 62


In [7]:
# Make a custom generator to format it the way we need.
from io import BytesIO
try:
    # py3
    from email.generator import BytesGenerator as TheGenerator
except ImportError:
    # py2
    from email.generator import Generator as TheGenerator

m1 = MIMEBase('text', 'plain')
m1.add_header('Content-disposition',
              'form-data; name="request-json"')
m1.set_payload(jsondata)
m2 = MIMEApplication(filedata,'octet-stream',encode_noop)
m2.add_header('Content-disposition',
              'form-data; name="file"; filename="%s"'%filename)
mp = MIMEMultipart('form-data', None, [m1, m2])

class MyGenerator(TheGenerator):
    def __init__(self, fp, root=True):
        # don't try to use super() here; in py2 Generator is not a
        # new-style class.  Yuck.
        TheGenerator.__init__(self, fp, mangle_from_=False,
                              maxheaderlen=0)
        self.root = root
    def _write_headers(self, msg):
        # We don't want to write the top-level headers;
        # they go into Request(headers) instead.
        if self.root:
            return
        # We need to use \r\n line-terminator, but Generator
        # doesn't provide the flexibility to override, so we
        # have to copy-n-paste-n-modify.
        for h, v in msg.items():
            self._fp.write(('%s: %s\r\n' % (h,v)).encode())
        # A blank line always separates headers from body
        self._fp.write('\r\n'.encode())

    # The _write_multipart method calls "clone" for the
    # subparts.  We hijack that, setting root=False
    def clone(self, fp):
        return MyGenerator(fp, root=False)

fp = BytesIO()
g = MyGenerator(fp)
#g.flatten(mp)
#data = fp.getvalue()
headers = {'Content-type': mp.get('Content-type')}

request = requests.post(url=uploadurl, headers=headers, data=mp.as_bytes())

AttributeError: 'dict' object has no attribute 'encode'

In [22]:
dir(mp)

['__bytes__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_charset',
 '_default_type',
 '_get_params_preserve',
 '_headers',
 '_payload',
 '_unixfrom',
 'add_header',
 'as_bytes',
 'as_string',
 'attach',
 'defects',
 'del_param',
 'epilogue',
 'get',
 'get_all',
 'get_boundary',
 'get_charset',
 'get_charsets',
 'get_content_charset',
 'get_content_disposition',
 'get_content_maintype',
 'get_content_subtype',
 'get_content_type',
 'get_default_type',
 'get_filename',
 'get_param',
 'get_params',
 'get_payload',
 'get_unixfrom',
 'is_multipart',
 'items',
 'keys',
 'policy',